<a href="https://colab.research.google.com/github/JayThibs/hyperdrive-vs-automl-plus-deployment/blob/main/automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated ML

Note: For data exploration, go to hyperparameter_tuning.ipynb

# Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


In [ ]:
%%writefile feature_preprocessing.py

import numpy as np
import pandas as pd

def bools(df):
    """
    public_meeting: we will fill the nulls as 'False'
    permit: we will fill the nulls as 'False
    """
    z = ['public_meeting', 'permit']
    for i in z:
        df[i].fillna(False, inplace = True)
        df[i] = df[i].apply(lambda x: float(x))
    return df

def locs(df, trans = ['longitude', 'latitude', 'gps_height', 'population']):
    """
    fill in the nulls for ['longitude', 'latitude', 'gps_height', 'population'] by using medians from 
    ['subvillage', 'district_code', 'basin'], and lastly the overall median
    """
    df.loc[df.longitude == 0, 'latitude'] = 0
    for z in trans:
        df[z].replace(0., np.NaN, inplace = True)
        df[z].replace(1., np.NaN, inplace = True)
        
        for j in ['district_code', 'basin']:
        
            df['median'] = df.groupby([j])[z].transform('median')
            df[z] = df[z].fillna(df['median'])
        
        df[z] = df[z].fillna(df[z].median())
        del df['median']
    return df

def construction(df):
    """
    A lot of null values for construction year. Of course, this is a missing value (a placeholder).
    For modeling purposes, this is actually fine, but we'll have trouble with visualizations if we
    compare the results for different years, so we'll set the value to something closer to
    the other values that aren't placeholders. Let's look at the unique years and set the null
    values to 50 years sooner.
    Let's set it to 1910 since the lowest "good" value is 1960.
    """
    df.loc[df['construction_year'] < 1950, 'construction_year'] = 1910
    return df

# Alright, now let's drop a few columns

def removal(df):
  # id: we drop the id column because it is not a useful predictor.
  # amount_tsh: is mostly blank - delete
  # wpt_name: not useful, delete (too many values)
  # subvillage: too many values, delete
  # scheme_name: this is almost 50% nulls, so we will delete this column
  # num_private: we will delete this column because ~99% of the values are zeros.
  features_to_drop = ['id','amount_tsh',  'num_private', 
          'quantity', 'quality_group', 'source_type', 'payment', 
          'waterpoint_type_group', 'extraction_type_group', 'wpt_name', 
          'subvillage', 'scheme_name']
  df = df.drop(features_to_drop, axis=1)

  return df

def dummy(df):
    dummy_cols = ['funder', 'installer', 'basin', 'lga', 'ward', 'public_meeting', 'recorded_by',
        'scheme_management', 'permit', 'extraction_type',
        'extraction_type_class', 'management', 'management_group',
        'payment_type', 'water_quality', 'quantity_group', 'source',
        'source_class', 'waterpoint_type', 'region']

    df = pd.get_dummies(df, columns=dummy_cols)

    return df

def dates(df):
    """
    date_recorded: this might be a useful variable for this analysis, although the year itself would be useless in a practical scenario moving into the future. We will convert this column into a datetime, and we will also create 'year_recorded' and 'month_recorded' columns just in case those levels prove to be useful. A visual inspection of both casts significant doubt on that possibility, but we'll proceed for now. We will delete date_recorded itself, since random forest cannot accept datetime
    """
    df['date_recorded'] = pd.to_datetime(df['date_recorded'])
    df['year_recorded'] = df['date_recorded'].apply(lambda x: x.year)
    df['month_recorded'] = df['date_recorded'].apply(lambda x: x.month)
    df['date_recorded'] = (pd.to_datetime(df['date_recorded'])).apply(lambda x: x.toordinal())
    return df

def dates2(df):
    """
    Turn year_recorded and month_recorded into dummy variables
    """
    for z in ['month_recorded', 'year_recorded']:
        df[z] = df[z].apply(lambda x: str(x))
        good_cols = [z+'_'+i for i in df[z].unique()]
        df = pd.concat((df, pd.get_dummies(df[z], prefix = z)[good_cols]), axis = 1)
        del df[z]
    return df

def small_n(df):
    "Collapsing small categorical value counts into 'other'"
    cols = [i for i in df.columns if type(df[i].iloc[0]) == str]
    df[cols] = df[cols].where(df[cols].apply(lambda x: x.map(x.value_counts())) > 100, "other")
    return df

## Dataset

### Overview

We'll be using the Pump it Up dataset from the DrivenData competition.

The description of the problem: 

> Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

In other words, our goal is to predict which water pumps are non-functioning or functioning, but in need of repair.

In this project, we will train a model using AutoML to train multiple multiple and choose the best performing model for deployment.

In [ ]:
# We loaded the dataset into Azure and we are grabbing it here.

from azureml.core import Workspace, Experiment, Dataset
from feature_preprocessing import *

# download config file in azure and put it in the current Notebooks folder
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Pump-it-Up-Data-Mining-the-Water-Table")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

# download config file in azure and put it in the current Notebooks folder
ws = Workspace.from_config()

dataset = Dataset.get_by_name(ws, name='Pump-it-Up-dataset')
X = dataset.to_pandas_dataframe()
y = X['status_group']
del X['status_group']

# Cleaning up the features of our dataset
X = bools(X)
X = locs(X)
X = construction(X)
X = removal(X)
X = dummy(X)
X = dates(X)
x = dates2(X)
X = small_n(X)

# Removing ">", "[" and "]" from the headers to make the data compatible with different algorithms (namely, xgboost)
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

# Converting the population values to log
X['population'] = np.log(X['population'])

# Splitting the dataset into a training and test set
# Test set will be used later
# The same random seed (42) for the Hyperdrive model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Concatenating the features and labels together to feed to our AutoML model
clean_train_df = pd.concat([X_train, y_train], axis=1)

Workspace name: quick-starts-ws-142186
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-142186


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Get the default datastore to be entered as a parameter in tabular dataset creation
datastore = workspace.get_default_datastore()

# Change pandas dataframe into a tabular dataset to be used in automl
training_data = TabularDatasetFactory.register_pandas_dataframe(clean_train_df, datastore, 'automl_data')

In [ ]:
training_data.take(5).to_pandas_dataframe()

# Setting up Experiment

We'll create a new experiment for our deployment of an AutoML model and create a project folder to hold the training scripts.

In [ ]:
experiment_name = 'automl-pump-it-up-operationalize'
project_folder = './automl-pipeline-project'

automl_experiment = Experiment(workspace, experiment_name)
automl_experiment

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Creating a compute cluster if there isn't one that is already created.

cpu_cluster_name = 'hypr-auto-clustr'

try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new computer target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_v2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

NameError: name 'workspace' is not defined

# AutoML Configuration

We'll create a new experiment for our deployment of an AutoML model and create a project folder to hold the training scripts.

Here we create the general AutoML settings object.


Calculate recall to test how well we do on True Positives. We can imagine a real scenario where we want to build a model that does not miss the non-functioning water pumps, and we care much less functioning water pumps that are incorrectly predicted as non-functional. Recall is useful to make sure we miss less True Positives.

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes": 20, # to set a limit on the amount of time AutoML will be running
    "max_concurrent_iterations": 5, # applies to the compute target we are using
    "primary_metric" : 'norm_macro_recall' # recall for our performance metric
}

# Setting AutoML config for model training.

automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification", # classifying if water pumps are functional
                             training_data=training_data, 
                             label_column_name="status_group", # our target variable for water pump function  
                             path = project_folder,
                             enable_early_stopping= True, # prevents automl from spending too much time on models that stopped improving, saves time and compute costs
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

NameError: name 'cpu_cluster' is not defined

## Create Pipeline and AutoMLStep

Defining the outputs for the AutoMLStep using TrainingOutput.

In [ ]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = workspace.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

NameError: name 'workspace' is not defined

## Create the AutoMLStep

In [ ]:
# Creating an AutoMLStep

automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True
    )

NameError: name 'automl_config' is not defined

In [ ]:
# Creating a Pipeline

from azureml.pipeline.core import Pipeline

pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=workspace,    
    steps=[automl_step])

NameError: name 'workspace' is not defined

In [ ]:
print('Submitting AutoML experiment...')

pipeline_run = automl_experiment.submit(pipeline)

Submitting AutoML experiment...


NameError: name 'automl_experiment' is not defined

# Run Details

Using the RunDeatils widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

NameError: name 'pipeline_run' is not defined

In [ ]:
pipeline_run.wait_for_completion()

# Examine Results

# Retrive the metrics of all child runs

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

# Best Model

In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

NameError: name 'pipeline_run' is not defined

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

NameError: name 'best_model_output' is not defined

In [ ]:
best_model.steps

NameError: name 'best_model' is not defined

# Test the model on the Test Set

In [ ]:
# Predict on the Test Set
ypred = best_model.predict(X_test)

# calculate recall
recall = recall_score(y_test, ypred, average='micro')
print('Recall: %.3f' % recall)

NameError: name 'best_model' is not defined

# Model Deployment

Registering the model, creating an inference config and deploy the model as a web service.

In other words, we are publishing the pipeline to enable a REST endpoint to rerun the pipeline from any HTTP library on any platform.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Pump it Up Train", description="Training Pump it Up pipeline", version="1.0")

published_pipeline

NameError: name 'pipeline_run' is not defined

Now we authenticate to retrieve the auth_header so that the endpoint can be used.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Test the Deployed Model

Here we will send a request to the deployed model to test it.



In [ ]:
import requests

# Geting the REST url from the endpoint property of the published pipeline
rest_endpoint = published_pipeline.endpoint

# Building an HTTP POST request to the endpoint
# We also add a JSON payload object with the experiment name
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

print(response.status_code)
print(response.elapsed)
print(response.json())

NameError: name 'published_pipeline' is not defined

In [ ]:
headers = {'Content-Type':'application/json'}
data = {"text": ['the food was horrible', 
                 'wow, this movie was truely great, I totally enjoyed it!',
                 'why the heck was my package not delivered in time?']}

resp = requests.post(aci_service.scoring_uri, json=data, headers=headers)
print("Prediction Results:", resp.json())

We are making it so that a request will trigger the run. We are also going to access the Id key from the response dict to get the value of the run id.

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

NameError: name 'rest_endpoint' is not defined

# Printing the logs of the web service

We can now use the run id to monitor the status of the new run. 

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

NameError: name 'ws' is not defined

# Printing the logs and Deleting the Service

In [ ]:
# Delete computer target in order to avoid incurring additional charges.

AmlCompute.delete(cpu_cluster)

NameError: name 'AmlCompute' is not defined